In [131]:
"""
this file takes a dataframe of recall and event data and clusters them by company name

refs
- https://medium.com/dnb-data-science-hub/company-name-matching-6a6330710334
- https://towardsdatascience.com/clustering-product-names-with-python-part-2-648cc54ca2ac/

prereq:
pip install scikit-learn nltk matplotlib cleanco levenshtein name_matching
"""

from get_data import get_dfs
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import re
import unicodedata
from cleanco import basename
import time
import Levenshtein
from name_matching.name_matcher import NameMatcher

nltk.download('stopwords')
nltk.download('punkt_tab')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prpar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\prpar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
# read dfs
df_recall, df_device_event, df_device = get_dfs()

loading table: recall....


C:\Users\prpar\Documents\OMSCS\s25 - cse6242 dva\CSE6242-Group-Project\priya-streamlit-test\get_data.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table}', conn)


                                     id cfres_id k_number pma_number  \
0  400cd411-6ba1-4360-9fea-b55e13468d00   212115     None       None   
1  1d35718a-6bc8-4706-bd3d-31741cd98324   212318     None       None   
2  224a2ab8-97ef-4799-879e-6083898ab2e7   211119     None       None   
3  b264b469-7c7f-48d5-bea0-57f38377569a   208478     None       None   
4  0aad8582-1a51-4374-998c-50cc0f4b4466   207500     None       None   

  event_date_initiated event_date_created event_date_posted  \
0           2025-01-15               None        2025-02-05   
1           2025-01-28               None        2025-02-03   
2           2024-08-08               None        2024-12-03   
3           2024-03-28               None        2024-07-01   
4           2024-04-16               None        2024-05-30   

  event_date_terminated     recall_status                   recalling_firm  \
0                  None  Open, Classified        Philips North America Llc   
1                  None  Open, C

C:\Users\prpar\Documents\OMSCS\s25 - cse6242 dva\CSE6242-Group-Project\priya-streamlit-test\get_data.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table}', conn)


                               event_id adverse_event_flag  \
0  137fc417-abd2-4c92-af98-f816d9d1b01c                  N   
1  7d7e75df-69d0-4344-a609-d8294fcaf9e1                  N   
2  b112bfb0-4306-498a-8ab9-76076641f4f9                  N   
3  ea373448-a219-4494-9cb6-2b3fbb403c8f                  N   
4  52eeaa61-643d-4452-aaa6-0b831af004c5                  N   

  date_facility_aware date_manufacturer_received date_of_event date_received  \
0                None                 2024-01-22    2024-01-22    2024-02-12   
1                None                 2024-01-18    2024-01-01    2024-02-12   
2                None                 2023-12-22    2023-12-22    2024-01-19   
3                None                 2024-01-24    2024-01-14    2024-02-12   
4          2024-02-19                       None          None    2024-03-15   

  date_report date_report_to_fda date_report_to_manufacturer  \
0  2024-02-12               None                        None   
1  2024-03-13     

C:\Users\prpar\Documents\OMSCS\s25 - cse6242 dva\CSE6242-Group-Project\priya-streamlit-test\get_data.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table}', conn)


                               event_id  \
0  137fc417-abd2-4c92-af98-f816d9d1b01c   
1  137fc417-abd2-4c92-af98-f816d9d1b01c   
2  7d7e75df-69d0-4344-a609-d8294fcaf9e1   
3  7d7e75df-69d0-4344-a609-d8294fcaf9e1   
4  b112bfb0-4306-498a-8ab9-76076641f4f9   

                                          brand_name catalog_number  \
0  T:SLIM X2 INSULIN PUMP WITH INTEROPERABLE TECH...        1005011   
1  T:SLIM X2 INSULIN PUMP WITH INTEROPERABLE TECH...        1005011   
2                        CONSTELLATION VISION SYSTEM     8065751150   
3                        CONSTELLATION VISION SYSTEM     8065751150   
4                                       AESPIRE 7100           None   

  date_received date_removed_flag date_returned_to_manufacturer  \
0    2024-02-12              None                          None   
1    2024-02-12              None                          None   
2    2024-02-12              None                          None   
3    2024-02-12              None             

In [209]:
print(df_recall.shape)
print(df_device_event.shape)
print(df_device.shape)

(54520, 38)
(100000, 78)
(199943, 32)


## preprocess relevant columns

From df_recall, we want to pull recalling_firm.
From df_device, we want to pull manufacturer_d_name.

In [68]:
def preprocess(df, col):
    """
    0. replace Nan with empty string
    1. lower
    2. remove non-ascii chars
    3. remove punctuation
    4. remove common legal business strings (like 'corp')
    """
    # remove NaN or None
    df = df.fillna('')
    
    stop_words = set(stopwords.words('english'))
    def f(x):
        x = x.lower() # step 1
        x = unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode() # step 2
        x = re.sub(r'[^\w\s]', '', x) # step 3
        x = basename(x) # step 4
        # word_tokens = word_tokenize(x) # step 5
        # filtered_text = [word for word in x if word.lower() not in stop_words] # 5
        # return ''.join(filtered_text) # 5
        return x
        
    return df[col].apply(lambda x: f(x))
    
    

In [69]:
df_recall['preproc_recalling_firm'] = preprocess(df_recall, 'recalling_firm')
df_device['preproc_manufacturer_d_name'] = preprocess(df_device, 'manufacturer_d_name')

In [162]:
# now we need to combine recall and event device manufacturers into one list before vectorizing
input_names = set(df_recall['preproc_recalling_firm'])
input_names.update(df_device['preproc_manufacturer_d_name'])
print(len(input_names))

4289


In [180]:
def vectorize_text(X, method='tfidf'):
    """
    convert text data into numerical representations.
    Possible methods:
    - bag of words
    - tf-idf
    """
    if method == 'tfidf':
        vectorizer = TfidfVectorizer(analyzer='word')
    else:
        vectorizer = CountVectorizer(analyzer='word')
    
    vectorizer.fit(X)
    return vectorizer.transform(X).toarray(), vectorizer # return vectorizer so it can be used later to assign cluster id to dfs
        
        

In [181]:
X, vectorizer = vectorize_text(input_names)
print(X.shape)


(4289, 5565)


## cluster using kmeans
General procedure:
1. take all manufacturers from manufacturer_d_name to create cluster based on similarity 
2. add a col which assigns a "label" or "cluster id" to similar names
3. apply cluster to manufacturers (recalling firms) in recall data
4. join the dfs on this cluster id


In [173]:
def cluster(X, k=1000, random_state=52):
    k = int(min(k, len(X) * 0.75))
    print(f'n_clusters={k}')
    
    kmeans = KMeans(n_clusters=k, random_state=random_state)
    clusters = kmeans.fit_predict(X)
    return clusters, kmeans



In [175]:
start = time.time()
df_device, kmeans = cluster(X)
print(f'------------ elapsed train time: {time.time() - start} seconds -------------')

n_clusters=1000
------------ elapsed train time: 14.659651279449463 seconds -------------


In [ ]:
######## WARNING : APPLYING K MEANS IS VERY TIME CONSUMING #####

In [ ]:
# apply clustering to both datasets. first vectorize by the preprocessed col and predict
def apply_cluster(x, vectorizer, kmeans):
    x = vectorizer.transform([x]).toarray()
    return kmeans.predict(x)


In [ ]:
####### transform recall data ########

In [189]:
# start = time.time()
# df_recall['cluster_id'] = df_recall['preproc_recalling_firm'].apply(lambda x: apply_cluster(x, vectorizer, kmeans))
# print(f'------------ elapsed transform time: {time.time() - start} seconds -------------')

In [ ]:
# print(df_recall['cluster_id'].value_counts())
# df_recall[['cluster_id', 'recalling_firm', 'preproc_recalling_firm']]

In [ ]:
####### transform event device data ########

In [ ]:
# start = time.time()
# df_device['cluster_id'] = df_device['preproc_manufacturer_d_name'].apply(lambda x: apply_cluster(x, vectorizer, kmeans))
# print(f'------------ elapsed transform time: {time.time() - start} seconds -------------')

In [ ]:
# print(df_device['cluster_id'].value_counts())
# df_device[['cluster_id', 'manufacturer_d_name', 'preproc_manufacturer_d_name']]

## cluster using distance metrics
Use distance metrics to find similar matches between the two dfs.

In [ ]:
# # create method to join lists by fuzzy matching (lowest levenshtein distance)
# def join_dfs_levenshtein(df1, df2, left_on, right_on, threshold, method='levenshtein'):
#     """
#     joins two dfs based on distance between specified columns. apply threshold to be considered match.
#     method options:
#     - levenshtein
#     - cosine similarity
#     - jaccard similarity
#     """
#     start = time.time()
    
#     # get unique values from each df
#     df1_unique = set(df1[left_on])
#     df2_unique = set(df2[right_on])

#     # iterate to get most similar matches
#     # merged_rows = []
#     # for _, row1 in df1.iterrows():
#     #     for _, row2 in df2.iterrows():
#     #         if method == 'levenshtein':
#     #             distance = Levenshtein.distance(str(row1[left_on]), str(row2[right_on]))
#     #             if distance <= threshold:
#     #                 merged_rows.append({**row1.to_dict(), **row2.to_dict(), 'levenshtein_distance': distance})

                    
#     print(f'------------ elapsed train time: {time.time() - start} seconds -------------')
#     return pd.DataFrame(merged_rows)


In [198]:
# threshold = 10
# merged_df = join_dfs_levenshtein(df_recall, df_device, 'preproc_recalling_firm', 'preproc_manufacturer_d_name', threshold)
# print(merged_df)

In [204]:
# use name_matching from medium: https://medium.com/dnb-data-science-hub/company-name-matching-6a6330710334
matcher = NameMatcher(top_n=3,
    # lowercase=True,
    punctuations=True,
    # remove_ascii=True,
    # legal_suffixes=False,
    # common_words=False,
    verbose=True)

matcher.set_distance_metrics(['discounted_levenshtein',
                              'SSK', 
                              'fuzzy_wuzzy_token_sort'])



In [220]:
# TODO - USE MATCHER ON UNIQUE NAMES ONLY
matcher.load_and_process_master_data('preproc_recalling_firm', df_recall)
matches = matcher.match_names(to_be_matched=df_device, column_matching='preproc_manufacturer_d_name')

preprocessing...

preprocessing complete 
 searching for matches...



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [03:11<00:00,  4.79s/it]


possible matches found   
 fuzzy matching...



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 199943/199943 [10:27<00:00, 318.39it/s]

done


In [232]:
# combine the dfs based on the matches
combined = pd.merge(df_recall, matches, left_index=True, right_on='match_index')
combined = pd.merge(combined, df_device, left_index=True, right_index=True)

In [241]:
combined[combined['score'] > 80][['match_name', 'score', 'recalling_firm', 'preproc_recalling_firm', 'manufacturer_d_name', 'preproc_manufacturer_d_name']]

,match_name,score,recalling_firm,preproc_recalling_firm,manufacturer_d_name,preproc_manufacturer_d_name
40896,depuy orthopaedics,100.000000,"Depuy Orthopaedics, Inc.",depuy orthopaedics,"DEPUY ORTHOPAEDICS, INC.",depuy orthopaedics
40897,depuy orthopaedics,100.000000,"Depuy Orthopaedics, Inc.",depuy orthopaedics,"DEPUY ORTHOPAEDICS, INC.",depuy orthopaedics
110433,depuy orthopaedics,100.000000,"Depuy Orthopaedics, Inc.",depuy orthopaedics,"DEPUY ORTHOPAEDICS, INC.",depuy orthopaedics
110435,depuy orthopaedics,100.000000,"Depuy Orthopaedics, Inc.",depuy orthopaedics,"DEPUY ORTHOPAEDICS, INC.",depuy orthopaedics
9681,biomerieux,100.000000,Biomerieux Inc,biomerieux,BIOMÉRIEUX SA,biomerieux
...,...,...,...,...,...,...
166574,becton dickinson,88.527472,Becton Dickinson & Co.,becton dickinson,BECTON DICKINSON AND CO.,becton dickinson and
174894,becton dickinson,88.527472,Becton Dickinson & Co.,becton dickinson,BECTON DICKINSON AND CO.,becton dickinson and
174895,becton dickinson,88.527472,Becton Dickinson & Co.,becton dickinson,BECTON DICKINSON AND CO.,becton dickinson and
177874,becton dickinson,88.527472,Becton Dickinson & Co.,becton dickinson,BECTON DICKINSON AND CO.,becton dickinson and


In [238]:
# IDEA - run this once and save as a df object
import pickle
try:
    combined.to_pickle('combined_df.pkl')
    # Save the model to a pickle file
    with open('kmeans.pkl', 'wb') as file:
        pickle.dump(kmeans, file)
    print('saved objects successfully!')
except Exception as e:
    print(e)


saved objects successfully!


## cluster using similarity ratio
Use SequenceMatcher on combined list of manufacturers to find similar groups. 
1. Combine dfs into two cols: [Recall/Event, 'Preproc Manufacturer Name'].
2. Apply similarity ratios to group similar strings. Label by "key"


<b>!! Has not run successfully !!</b>

In [216]:
# from difflib import SequenceMatcher
# from itertools import groupby

# def group_similar_strings(strings, threshold=0.8):
#     """
#     groups similar strings by using SequenceMatcher
#     """
#     start = time.time()
    
#     def similarity_ratio(a, b):
#         return SequenceMatcher(None, a, b).ratio()

#     sorted_strings = sorted(strings)

#     # group by similarity to the first string in each group
#     grouped_strings = {}
#     for key, group in groupby(sorted_strings, lambda s: sorted([x for x in sorted_strings if similarity_ratio(s,x) >= threshold])[0]):
#         grouped_strings[key] = list(group)

#     print(f'------------ elapsed train time: {time.time() - start} seconds -------------')
#     return grouped_strings


In [218]:
# # now we need to combine recall and event device manufacturers into one list before vectorizing
# # input_names = set(df_recall['preproc_recalling_firm'])
# # input_names.update(df_device['preproc_manufacturer_d_name'])
# # print(len(input_names))

# grouped_strings = group_similar_strings(input_names)
# print(grouped_strings)
